In [1]:
import sympy as sp
import numpy as np
sp.init_printing()

%load_ext cython

In [2]:
x0, x1, x2 = sp.symbols("x0, x1, x2")
params = sp.MatrixSymbol('params', 3, 1)

In [3]:
A = sp.Matrix([x0,
               x1, 
               x2, 
               x2**2+x0+x1, 
               x1**2+x1+1, 
               x1**2+x0+1, 
               x2+1, 
               x0**2+x1+x2, 
               x2**2+x2+1, 
               x2**2+x0**2+x0+2, 
               x1+1, 
               x0**2+x0+x1])

In [4]:
param_map = dict(zip([x0, x1, x2], params))
B = A.xreplace(param_map)
R = sp.MatrixSymbol("coeff", B.shape[0], B.shape[1])

In [5]:
from sympy.printing.ccode import C99CodePrinter
printer = C99CodePrinter()

class CustomCodePrinter(C99CodePrinter):
    def _print_Pow(self, expr):
        if expr.exp.is_integer and expr.exp > 0 and expr.exp < 5:
            return '*'.join([self._print(expr.base) for i in range(expr.exp)])
        else:
            return super()._print_Pow(expr)

In [6]:
from sympy.utilities.codegen import codegen, default_datatypes

In [7]:
sp.Eq(R,B)

        ⎡              params₀₀              ⎤
        ⎢                                    ⎥
        ⎢              params₁₀              ⎥
        ⎢                                    ⎥
        ⎢              params₂₀              ⎥
        ⎢                                    ⎥
        ⎢                                2   ⎥
        ⎢  params₀₀ + params₁₀ + params₂₀    ⎥
        ⎢                                    ⎥
        ⎢              2                     ⎥
        ⎢      params₁₀  + params₁₀ + 1      ⎥
        ⎢                                    ⎥
        ⎢                         2          ⎥
        ⎢      params₀₀ + params₁₀  + 1      ⎥
        ⎢                                    ⎥
coeff = ⎢            params₂₀ + 1            ⎥
        ⎢                                    ⎥
        ⎢          2                         ⎥
        ⎢  params₀₀  + params₁₀ + params₂₀   ⎥
        ⎢                                    ⎥
        ⎢              2                     ⎥
        ⎢    

In [8]:
from sympy.codegen.ast import real, float32
customprinter = CustomCodePrinter()
#customprinter.type_aliases[real] = float32 # cosf instead of cos
default_datatypes["float"].cname = "float" # float instead of double

[(cf, cs), (hf, hs)] = codegen(('coefficients', sp.Eq(R,B)), language='c', printer=customprinter)
print(cs)

/******************************************************************************
 *                       Code generated with sympy 1.4                        *
 *                                                                            *
 *              See http://www.sympy.org/ for more information.               *
 *                                                                            *
 *                       This file is part of 'project'                       *
 ******************************************************************************/
#include "coefficients.h"
#include <math.h>

void coefficients(float *params, float *coeff) {

   coeff[0] = params[0];
   coeff[1] = params[1];
   coeff[2] = params[2];
   coeff[3] = params[0] + params[1] + params[2]*params[2];
   coeff[4] = params[1]*params[1] + params[1] + 1;
   coeff[5] = params[0] + params[1]*params[1] + 1;
   coeff[6] = params[2] + 1;
   coeff[7] = params[0]*params[0] + params[1] + params[2];
   coeff[8] = params

In [9]:
codegen(('coefficients', sp.Eq(R,B)), language='c', printer=customprinter, prefix='coefficients', to_files=True)

In [10]:
%%writefile cy_coefficients.pyxbld
import numpy

#            module name specified by `%%cython_pyximport` magic
#            |        just `modname + ".pyx"`
#            |        |
def make_ext(modname, pyxfilename):
    from setuptools.extension import Extension
    return Extension(modname,
                     sources=[pyxfilename, 'coefficients.c'],
                     include_dirs=['.', numpy.get_include()])

Overwriting cy_coefficients.pyxbld


In [11]:
%%cython_pyximport cy_coefficients
import numpy as np
cimport numpy as cnp # cimport gives us access to NumPy's C API

# here we just replicate the function signature from the header
cdef extern from "coefficients.h":
    void coefficients(float *params, float *result)

# here is the "wrapper" signature that conforms to the odeint interface
def cy_coefficients(cnp.ndarray[cnp.float32_t, ndim=1] params, size):
    # preallocate our output array
    cdef cnp.ndarray[cnp.float32_t, ndim=1] result = np.empty(size, dtype='float32')
    # now call the C function
    coefficients(<float *> params.data, <float *> result.data)
    # return the result
    return result

/nfs/ka/home/freibold/.local/lib/python3.7/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /nfs/ka/home/freibold/OneDrive/MotionCurve/cy_coefficients.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [12]:
params = np.array([1, 2, 3], dtype='float32')
result = cy_coefficients(params, 12)
print(result)

[ 1.  2.  3. 12.  7.  6.  4.  6. 13. 13.  3.  4.]


In [13]:
A.subs({(x0, 1), (x1, 2), (x2, 3)})

⎡1 ⎤
⎢  ⎥
⎢2 ⎥
⎢  ⎥
⎢3 ⎥
⎢  ⎥
⎢12⎥
⎢  ⎥
⎢7 ⎥
⎢  ⎥
⎢6 ⎥
⎢  ⎥
⎢4 ⎥
⎢  ⎥
⎢6 ⎥
⎢  ⎥
⎢13⎥
⎢  ⎥
⎢13⎥
⎢  ⎥
⎢3 ⎥
⎢  ⎥
⎣4 ⎦